In [9]:
#====================================================================================================
# kogpt2 를 이용한 summarize 예시
# => 
# => https://www.nbshare.io/notebook/764386829/Amazon-Review-Summarization-Using-GPT-2-And-PyTorch/
#
# => text generation 모델이므로  acc 구하는 것은 의미 없음(*따라서 train loss, val loss 만 구함)
#
# [훈련 dataset]
# => input_ids = <s>text</s>
# => labels = input_ids와 동일
# => attiention_mask
#====================================================================================================

import torch
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler 
import numpy as np
import pandas as pd
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
from transformers import AdamW, get_linear_schedule_with_warmup

from tqdm.notebook import tqdm
import os
import time
from myutils import GPU_info, seed_everything, mlogging, SaveBERTModel, AccuracyForMLM
from summarizer import TransformerSummarizer
model_path='../model/gpt-2/kogpt-2/'
#model_path='skt/kogpt2-base-v2'
#model_path = "gpt2-medium"

# 출력
OUTPATH = '../model/gpt-2/kogpt-2-ft-summarizer-0504/'

device = GPU_info()
print(device)

#seed 설정
seed_everything(222)

#logging 설정
logger =  mlogging(loggername="gpt2-ft", logfilename="../log/gpt2-ft")

True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0
logfilepath:../log/gpt2-ft_2022-05-04.log


In [2]:
# 요약문 corpus 파일 열기 
corpus_path = "../korpora/mycorpus/newspaper.csv"
with open(corpus_path, "r") as reviews_raw:
    reviews = reviews_raw.readlines()

In [3]:
reviews[:3]

['충주시는 민간보조사업의 증가와 보조금 집행관리에 대한 부당 행위가 증가함에따라 15일부터 25일까지 보조금 실태를 파악한 후 8월15일까지 세부감사를 진행  운영실태 전반에 대한 자체 감사를 실시할 계획이라고 밝혔다 ,충주시  민간지원 보조사업 대형축제 운영 감사 돌입\n',
 '국무조정실은 8일 오후 대전시청에서  대전지역 규제혁신 현장간담회 를 열고 대전과 충남에 취약한 뿌리산업 육성방안으로 규제개선을 논의하였으며  관계자는 기업과 국민들이 체감할 수 있는 규제개선이 이루어지도록 최대한 노력할 것이라고 밝혔다 ,대전도 뿌리산업 특화단지 길 열린다   국무조정실 규제개선키로\n',
 '중국 경제일간지 21세기경제보도는 중국 대형 생명보험사인 차이나라이프가  차이나라이프 중흥 이라는 전략적 목표를 세우고 비즈니스 중심의 조직 시스템 구축과 인터넷 생명보험회사 설립에 착수 중이라고 17일 보도하였다 ,중국 생보사 차이나라이프  인터넷 보험사 설립 추진\n']

In [4]:
print(len(reviews))

3091


In [5]:
# reviews 문장은 text, 요약문 식으로 되어 있다.
# => 이 구분자 , 를 <summarize> 로 변경함.
reviews = [review.replace(",", "<summarize>") for review in reviews]

In [6]:
reviews[1]

'국무조정실은 8일 오후 대전시청에서  대전지역 규제혁신 현장간담회 를 열고 대전과 충남에 취약한 뿌리산업 육성방안으로 규제개선을 논의하였으며  관계자는 기업과 국민들이 체감할 수 있는 규제개선이 이루어지도록 최대한 노력할 것이라고 밝혔다 <summarize>대전도 뿌리산업 특화단지 길 열린다   국무조정실 규제개선키로\n'

In [7]:
# 총 단어의 길이를 얻어옴
avg_length = sum([len(review.split()) for review in reviews])/len(reviews)
avg_length

35.62924619864122

In [17]:
# 총 단어의 길이보다 길게 설정
max_length = 100

In [10]:
# tokenizer 로딩 
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path,
                                                   bos_token='</s>',
                                                   eos_token='</s>',
                                                   unk_token='<unk>',
                                                   pad_token='<pad>',
                                                   mask_token='<mask>')

In [11]:
# 모델 정의 하고, embedding size를 tokenizer 사이즈만큼 조정
model = GPT2LMHeadModel.from_pretrained(model_path)

model.resize_token_embeddings(len(tokenizer))
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [13]:
# 구분자 <summarize> 토큰의 길이를 얻어옴
print(tokenizer.encode("<summarize>"))
extra_length = len(tokenizer.encode("<summarize>"))
print(f'***<summarize> token len:{extra_length}')


[9724, 457, 459, 10473, 9837, 21049, 443, 405]
***<summarize> token len:8


In [25]:
# dataset 설정 함수
class summarizeDataset(Dataset):
    def __init__(self, tokenizer, sentences, max_len, summarize_token_len):
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.eos = self.tokenizer.eos_token
        self.eos_id = self.tokenizer.eos_token_id
        self.sentences = sentences
        self.summarize_token_len = summarize_token_len
        self.result = []
        
        for sentence in self.sentences:
            # 한 문장 뒤에 </s>(EOS 토큰) 추가
            tokenized = self.tokenizer.encode(sentence + self.eos)
            #print(tokenized)
            # padd 
            padded = self.pad_truncate(tokenized)
            
            # 출력
            self.result.append(torch.tensor(padded))
           
    def __len__(self):
        return len(self.result)
    
    def __getitem__(self, item):
        return self.result[item]
    
    # padd 붙이는 함수
    def pad_truncate(self, name):
        
        name_length = len(name) - self.summarize_token_len
        
        if name_length < self.max_len:
            difference = self.max_len - name_length
            result = name + [self.eos_id] * difference
        elif name_length > self.max_len:
            result = name[:self.max_len + 3]+[self.eos_id]
        else:
            result = name
        
        return result
        

In [26]:
# dataset 만듬
dataset = summarizeDataset(tokenizer, reviews, max_length, extra_length)

In [27]:
dataset[1]

tensor([14932, 13305, 16428, 16308, 21734, 13899,  7888, 29718,   739, 13899,
         9635, 17143, 25002, 16380,  6826,  7191,  8765, 36055, 20759,  9026,
        12391, 27775,  8022,  9499, 16455, 13961, 10258, 16891,  7607, 21609,
        17143,  6841, 10402, 13564, 15423,   739,  9804,  9599, 50865,  9888,
         9136, 48262,  8705,  9025,  9080, 17143,  6841, 10886,  9442, 25682,
        19226, 10805,  8705, 11839, 28296,  7182,  9724,   457,   459, 10473,
         9837, 21049,   443,   405, 10238,  7235, 13961, 10258,  9125,  8756,
        18081,  9367, 13363,  7182,   739,   739, 14932, 13305,  7892, 17143,
        32931,  8511,  7426,   375,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1])

In [28]:
# 데이터 로더 생성 
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)

In [ ]:
#GPT2_model = TransformerSummarizer(transformer_type="GPT2", transformer_model_key=model_path)
#full = ''.join(GPT2_model(body, min_length=60))
#print(full)